## Model Data using Cassandra


### Please just submit this notebook in the Submission and make sure each cell has been executed and output is clearly displayed.

### The aim of the project is to solve the three queries given below.

### Introduction

There is a music streaming app called SoundCloud, that has been using their music streaming app and collecting data on songs and user activity and their aim is to analyze this data especially understanding what songs users are listening to. Currently, they are not making use of a NoSQL db and they have the data stored as a CSV file, thus its difficult for them to query the data. So our task is to create a NoSQL database for helping them with the analysis.

#### Import Packages 

I have used a cloud service (Datastax Astra) to host my database and it requires to use cassandra-driver package

In [3]:
! pip install cassandra-driver

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 33.4 MB/s eta 0:00:00


In [2]:
import json

id =""
sec=""
with open("/content/sample_data/cassandra.json", "r") as f:
  data = json.load(f)
  id = data['clientID']
  sec = data['clientSecret']

In [4]:
import pandas as pd
import numpy as np
import cassandra
import csv


##  The image below is a screenshot of what the data appears like in the event_data.csv

<img src="event_data_image.jpg">

#### Creating a Cluster

In [12]:
# Task: Make a connection to the cassandra instance on your local machine(127.0.0.1) and 
# create a session to establish connection and begin executing queries

from cassandra.cluster import Cluster
from cassandra.auth import PlainTextAuthProvider

cloud_config= {
  'secure_connect_bundle': '/content/sample_data/cassandra bundle.zip'
}

auth_provider = PlainTextAuthProvider(id, sec)
cluster = Cluster(cloud=cloud_config, auth_provider=auth_provider)
session = cluster.connect()

ERROR:cassandra.connection:Closing connection <LibevConnection(140497358641808) de1b27fd-ae97-4407-8ae9-0a124dbcb0f8-us-east1.db.astra.datastax.com:29042:e98e9571-aa43-4e97-bfcd-286514bd492b> due to protocol error: Error from server: code=000a [Protocol error] message="Beta version of the protocol used (5/v5-beta), but USE_BETA flag is unset"


#### Create & Set Keyspace

In [13]:
# Task: Create a Keyspace and Set KEYSPACE to the keyspace specified above
keyspace = 'soundcloud'
try:
  # session.execute("""
  #   CREATE KEYSPACE IF NOT EXISTS soundcloud
  #   WITH replication = {'class': 'SimpleStrategy', 'replication_factor': 1};
  # """)
  session.set_keyspace(keyspace)
except Exception as err:
  print(err)

  

I am using DataStax Astra so I set my keyspace while i created my account. As a free tier i cannot rename it modify it. 

## List of Queries 

### 1. Find the artist_name, song_title and length of song the SoundCloud app history that was heard during  session_number = 338, and item_in_session_number  = 4


### 2. Find the artist_name, song_title (sorted by item_in_session_number) and name(fname and lname) of the user for user_id = 10, session_number = 182
    

### 3. Find every name(fname and lname) of the user from the SoundCloud app history that listened to the song_title 'All Hands Against His Own'




### Query1 Table1: How should we model this data? Think about what should be our Primary Key/Partition Key/Clustering Key

In [14]:
## Task: Query 1: Find the artist_name, song_title and length of song the SoundCloud app history 
## that was heard during session_number = 338, and item_in_session_number = 4
## make use of create table command        

try:
  session.execute("""
    CREATE TABLE soundcloud_history (
      session_number INT,
      item_in_session_number INT,
      artist_name TEXT,
      song_title TEXT,
      length_of_song FLOAT,
      PRIMARY KEY ((session_number, item_in_session_number), artist_name, song_title)
    )
  """)
except Exception as err:
  print(err)

# SELECT artist_name, song_title, length_of_song
# FROM soundcloud_history
# WHERE session_number = 338 AND item_in_session_number = 4;

Table 'soundcloud.soundcloud_history' already exists


### Let's insert our data into of table

In [16]:

file_name = '/content/sample_data/event_data.csv'

df = pd.read_csv(file_name)
rows = df.iterrows()

print(df.shape)
print(df.columns)

for index, row in rows:
  try:
    query = (
        f"INSERT INTO soundcloud_history (session_number, item_in_session_number, artist_name, song_title, length_of_song) "
        f"VALUES (%s, %s, %s, %s, %s)"
    )
    params = (row['session_number'], row['item_in_session_number'], row['artist_name'], row['song_title'], row['length'])
    session.execute(query, params)
  except Exception as err:
    print(err)
    print(query)

# We have provided part of the code to set up the CSV file. Please complete the Apache Cassandra code below#
# with open(file_name, encoding = 'utf8') as f:
#     csv_reader = csv.reader(f)
#     next(csv_reader) # skip the header in the csv file
#     for i in range(0,5):
#       print(csv_reader[i])
#     for (index,row) in enumerate(csv_reader):
# ## Task: Write the INSERT statements and assign it to the query variable
#         query = "<Place your insert statement and assign the values here>"
        
#         ## Task: Match the column in the csv file to the column in the INSERT statement.
#         ## e.g., if you want to INSERT gender from csv file into the database you will use row[2]
#         ## e.g., if you want to INSERT location from csv file into database you will use row[7]
#         session.execute(query, (row[], row[]))

(6820, 11)
Index(['artist_name', 'fname', 'gender', 'item_in_session_number', 'lname',
       'length', 'level', 'location', 'session_number', 'song_title',
       'user_id'],
      dtype='object')


### Validate our Data Model using a SELECT

In [17]:
## Task: Make use of the SELECT statement and for loop to check if your query works and display the results
try:
  query = """
    SELECT artist_name, song_title, length_of_song 
    FROM soundcloud_history 
    WHERE session_number = 338 and item_in_session_number = 4
  """
  rows = session.execute(query)

  for row in rows:
    print(row)
except Exception as err:
  print(err)

Row(artist_name='Faithless', song_title='Music Matters (Mark Knight Dub)', length_of_song=495.30731201171875)


### Query2 Table2: How should we model this data? Think about what should be our Primary Key/Partition Key/Clustering Key 

In [18]:
## Task: Query 2: Find the artist_name, song_title (sorted by item_in_session_number) and 
## name(fname and lname) of the user for user_id = 10, session_number = 182 
## make use of create table command     

try:
  query = """
    CREATE TABLE IF NOT EXISTS soundcloud_history2 (
      user_id INT,
      session_number INT,
      item_in_session_number INT,
      artist_name TEXT,
      song_title TEXT,
      fname TEXT,
      lname TEXT,
      PRIMARY KEY ((user_id, session_number), item_in_session_number)
    );
  """
  session.execute(query)
except Exception as err:
  print(err)

### Let's insert our data into of table

In [19]:
# We have provided part of the code to set up the CSV file. Please complete the Apache Cassandra code below#
# file_name = 'event_data.csv'

# with open(file_name, encoding = 'utf8') as f:
#     csv_reader = csv.reader(f)
#     next(csvreader) # skip the header in the csv file
#     for row in csv_reader:
# ## Task: Write the INSERT statements and assign it to the query variable
#         query = "<Place your insert statement and assign the values here>"
        
#         ## Task: Match the column in the csv file to the column in the INSERT statement.
#         ## e.g., if you want to INSERT gender from csv file into the database you will use row[2]
#         ## e.g., if you want to INSERT location from csv file into database you will use row[7]
#         session.execute(query, (row[], row[]))


file_name = '/content/sample_data/event_data.csv'

df = pd.read_csv(file_name)

for index, row in  df.iterrows():
  try:
    query = (
        f"INSERT INTO soundcloud_history2 (user_id, session_number, item_in_session_number, artist_name, song_title, fname, lname) "
        f"VALUES (%s, %s, %s, %s, %s, %s, %s)"
    )
    params = (row['user_id'], row['session_number'], row['item_in_session_number'], row['artist_name'], row['song_title'], row['fname'], row['lname'])
    session.execute(query, params)
  except Exception as err:
    print(err)
    print(query)

### Validate our Data Model using a SELECT

In [20]:
## Task: Make use of the SELECT statement and for loop to check if your query works and display the results
try:
  query = """
    SELECT artist_name, song_title, fname, lname
    FROM soundcloud_history2 
    WHERE user_id = 10 AND session_number = 182 
    ORDER BY item_in_session_number;
  """
  rows = session.execute(query)
  for row in rows:
    print(row)
except err:
  print(err)

Row(artist_name='Down To The Bone', song_title="Keep On Keepin' On", fname='Sylvie', lname='Cruz')
Row(artist_name='Three Drives', song_title='Greece 2000', fname='Sylvie', lname='Cruz')
Row(artist_name='Sebastien Tellier', song_title='Kilometer', fname='Sylvie', lname='Cruz')
Row(artist_name='Lonnie Gordon', song_title='Catch You Baby (Steve Pitron & Max Sanna Radio Edit)', fname='Sylvie', lname='Cruz')


### Query3 Table3: How should we model this data? Think about what should be our Primary Key/Partition Key/Clustering Key

In [21]:
## Task: Query 3: Find every name(first and lastname) of the user from the SoundCloud app history that listened 
## to the song_title 'All Hands Against His Own'
## make use of create table command                   

query = """
  CREATE TABLE IF NOT EXISTS app_history (
      song_title text,
      user_id int,
      fname text,
      lname text,
      PRIMARY KEY (song_title, user_id)
  ) WITH CLUSTERING ORDER BY (user_id ASC);
"""

try:
  session.execute(query)
except err:
  print(err)

### Let's insert our data into of table

In [22]:
# We have provided part of the code to set up the CSV file. Please complete the Apache Cassandra code below#
# file_name = 'event_data.csv'

# with open(file_name, encoding = 'utf8') as f:
#     csv_reader = csv.reader(f)
#     next(csvreader) # skip the header in the csv file
#     for row in csv_reader:
# ## Task: Write the INSERT statements and assign it to the query variable
#         query = "<Place your insert statement and assign the values here>"
        
#         ## Task: Match the column in the csv file to the column in the INSERT statement.
#         ## e.g., if you want to INSERT gender from csv file into the database you will use row[2]
#         ## e.g., if you want to INSERT location from csv file into database you will use row[7]
#         session.execute(query, (row[], row[]))


file_name = '/content/sample_data/event_data.csv'

df = pd.read_csv(file_name)

for index, row in  df.iterrows():
  try:
    query = (
        f"INSERT INTO app_history (song_title, user_id, fname, lname) "
        f"VALUES (%s, %s, %s, %s)"
    )
    params = (row['song_title'], row['user_id'], row['fname'], row['lname'])
    session.execute(query, params)
  except Exception as err:
    print(err)
    print(query)

### Validate our Data Model using a SELECT

In [23]:
## Task: Make use of the SELECT statement and for loop to check if your query works and display the results
try:
  query = """
  SELECT fname, lname from app_history
  WHERE song_title='All Hands Against His Own'
  """
  rows = session.execute(query)
  for row in rows:
    print(row)
except Exception as err:
  print(err)

Row(fname='Jacqueline', lname='Lynch')
Row(fname='Tegan', lname='Levine')
Row(fname='Sara', lname='Johnson')


### Drop the tables before closing out the sessions

In [24]:
query = "DROP TABLE soundcloud_history"

session.execute(query)

### Close the session and cluster connection¶

In [11]:
cluster.shutdown()
session.shutdown()